In [3]:
import torch
import torch.nn as nn 
from torch.nn import functional as F

In [39]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# print(device)

block_size = 8
batch_size = 4
learning_rate = 3e-3
max_iters = 1000
eval_iters = 250
dropout = 0.2
n_embd = 384
n_layer= 4

In [19]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)

In [21]:
string_to_text=  {ch:i for i , ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_text[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, )) #take random index from 0 to block size
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device) ,y.to(device)
    return x, y


In [29]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train', 'val']:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [45]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=  n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        #FF-NN
        self.ffwd = FeedForward(n_embd)
        #add + norm, post-norm architecture /\ (X + Y) , Y -> F(X)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #hidden D-layer(s)
        self.ln_f = nn.LayerNorm(n_embd) #layer normalisation
        self.lm_head = nn.Linear(n_embd, vocab_size) #linear transformation

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)

        tok_emb = self.token_embedding_table(idx) # B , T , C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T,C 
        x= tok_emb + pos_emb # B, T, C
        x = self.blocks(x) # B, T, C
        x = self.ln_f(x) # B, T,C 
        logits = self.lm_head(x) # B, T, VOCAB_SIZE

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            #BATCH, TIME / SEQ LENGTH, VOCAB SIZE / CHANNEL
            logits = logits.view(B*T, C)
            targets= targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[: , - 1, :] #becomes (B,C), last time sample
            probs = F.softmax(logits, dim=-1) #(B, C) where vocab dimension=[:-1]
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1 )
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

    def getEmbedding(self):
        return self.token_embedding_table

model = GPTLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long, device=device) 
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)
# generated_chars_raw = m.generate(context, max_new_tokens=500)
# print(generated_chars_raw)



NameError: name 'Block' is not defined

In [33]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter} , train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    #sample batch data
    xb, yb = get_batch('train') #in, ot

    #eval loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none = True) #enable for RNNs
    loss.backward()
    optimiser.step()
print(loss.item())

step: 0 , train loss: 4.809, val loss: 4.830
step: 250 , train loss: 4.241, val loss: 4.227
step: 500 , train loss: 3.741, val loss: 3.746
step: 750 , train loss: 3.370, val loss: 3.408
3.0001449584960938
